In [25]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [26]:
import logging
import seaborn as sns
import pandas as pd
import awswrangler as wr
import matplotlib.pyplot as plt
from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url

In [27]:
customer = "Junglueck"
audience ="66d17985c98dbc05719f6ebe"
url = return_api_url()
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]
path_to_save = f"/Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/SprintStories/EN-2669-incident-action-probs/"

In [28]:
file_name ="targeting_history.csv"
try:
    df = pd.read_csv(f"{path_to_save}{file_name}")
except FileNotFoundError:
    df = pd.DataFrame()
    for date in pd.date_range(start="2025-07-01", end="2025-07-31", freq="D"):
        date = date.strftime("%Y%m%d")
        try:
            temp = wr.s3.read_parquet(f"s3://{account_id}/targeting.history/{date}/{audience}.parquet")
        except wr.exceptions.NoFilesFound:
            print(f". No data for {date}")
            continue
        temp["bucket_date"] = date
        df = pd.concat([df, temp])
    df = df.reset_index()
    df.to_csv(f"{path_to_save}{file_name}", index=False)
df

In [29]:
sns.lineplot(data=df, x="bucket_date", y="counterfactual")
plt.grid(True)
plt.xticks(rotation=90)
plt.ylim(0, df["counterfactual"].max()+0.2)

In [45]:
means = df.groupby("bucket_date")["counterfactual"].mean().reset_index().rename(columns={"counterfactual": "mean", "bucket_date":"date"})
means

In [46]:
def detect_negative_trends(df, threshold=3):
      """Detect significant negative trends (> 3% decrease) in a DataFrame"""
      df = df.sort_values('date').copy()

      # Calculate percentage change from previous period
      df['pct_change'] = df['mean'].pct_change() * 100

      # Flag significant negative trends
      df['significant_negative'] = df['pct_change'] < -threshold

      return df

means = detect_negative_trends(means)
if means["significant_negative"].sum() > 0:
      print("Significant drop in probabilities")
      drop_values = means[means["significant_negative"] == True]["pct_change"].tolist()
      print(drop_values)
else:
      print("No significant drop in probabilities")



In [50]:
means[means["significant_negative"] == True]